# 📑 Homework #4 – "From VEP JSON to a Relational DB"

## Context
The **Ensembl Variant Effect Predictor (VEP)** outputs rich JSON annotations for each variant observed in a sample.  
Bioinformatic workflows often need to store those annotations in a **relational database** so they can be queried efficiently (e.g. *"show all LOF variants in **TP53** with genotype 0/1 in any sample"*).

You are given **three VEP JSON files** – `HG001.json`, `HG002.json`, `HG003.json` – one per sample.  
Each file is an **array** of variant objects.

Below is an actual VEP JSON variant object structure:
```json
{
  "input": "chr1\t14599\t.\tT\tA\t44.3\tPASS\t.\tGT:GQ:DP:AD:VAF:PL\t1/1:5:3:1,2:0.666667:42,3,0",
  "id": ".",
  "transcript_consequences": [
    {
      "gene_id": "ENSG00000223972",
      "hgvsg": "chr1:g.14599T>A",
      "source": "Ensembl",
      "given_ref": "T",
      "transcript_id": "ENST00000450305",
      "impact": "MODIFIER",
      "gene_symbol_source": "HGNC",
      "strand": 1,
      "distance": 929,
      "canonical": 1,
      "used_ref": "T",
      "hgnc_id": "HGNC:37102",
      "consequence_terms": ["downstream_gene_variant"],
      "gene_symbol": "DDX11L1",
      "variant_allele": "A"
    },
    {
      "canonical": 1,
      "variant_allele": "A",
      "strand": 1,
      "gene_symbol": "DDX11L2",
      "distance": 190,
      "consequence_terms": ["downstream_gene_variant"],
      "impact": "MODIFIER",
      "gene_symbol_source": "EntrezGene",
      "used_ref": "T",
      "transcript_id": "ENST00000456328",
      "given_ref": "T",
      "source": "Ensembl",
      "hgvsg": "chr1:g.14599T>A",
      "gene_id": "ENSG00000290825"
    },
    {
      "hgnc_id": "HGNC:38034",
      "consequence_terms": ["intron_variant", "non_coding_transcript_variant"],
      "gene_symbol": "WASH7P",
      "variant_allele": "A",
      "used_ref": "T",
      "impact": "MODIFIER",
      "gene_symbol_source": "HGNC",
      "intron": "10/10",
      "strand": -1,
      "hgvsc": "ENST00000488147.1:n.1254-98A>T",
      "canonical": 1,
      "gene_id": "ENSG00000227232",
      "hgvsg": "chr1:g.14599T>A",
      "source": "Ensembl",
      "given_ref": "T",
      "transcript_id": "ENST00000488147"
    }
  ],
  "start": 14599,
  "most_severe_consequence": "non_coding_transcript_exon_variant",
  "end": 14599,
  "strand": 1,
  "seq_region_name": "chr1",
  "assembly_name": "GRCh38",
  "allele_string": "T/A",
  "custom_annotations": {
    "alfa": [
      {
        "fields": {
          "AF_oas": 0,
          "AF_eas": 0,
          "AF_afr": 0,
          "AN_eas": 56,
          "AN_len": 292,
          "AF_len": 0,
          "AN": 11688,
          "AN_lac": 74,
          "AN_afa": 2230,
          "AF_afo": 0,
          "AF_otr": 0.00359712230215827,
          "FILTER": ".",
          "AF_eur": 0.0730062776098582,
          "AF": 0.0538158795345654,
          "AF_asn": 0,
          "AN_otr": 278,
          "AN_afo": 102,
          "AN_sas": 36,
          "AN_asn": 74,
          "AN_oas": 18,
          "AN_eur": 8602,
          "AF_afa": 0,
          "AF_sas": 0,
          "AF_lac": 0,
          "AN_afr": 2332
        },
        "allele": "A",
        "name": "1:14599-14599"
      }
    ]
  }
}
```

## Tasks

1. **Design a PostgreSQL schema** with exactly these three tables:

| Table         | Primary Key  | Columns                                                                   | Foreign Keys          |
|---------------|--------------|---------------------------------------------------------------------------|-----------------------|
| **gene**      | gene_id      | symbol, biotype                                                          | –                     |
| **variant**   | variant_id   | chrom, pos, ref, alt, rs_id (nullable), impact (best transcript)         | gene_id → gene        |
| **variant_call** | vc_id      | sample_name, genotype, depth (DP)                                          | variant_id → variant  |

2. **Write a Python 3 script** that:
   * Accepts the three JSON filenames as CLI arguments
   * Parses each JSON file (arrays of variant objects)
   * Extracts sample names from filenames (e.g., `HG001`, `HG002`, `HG003`)
   * Implements transcript selection logic (canonical → worst impact)
   * Inserts or updates rows via **SQLAlchemy ORM** into a PostgreSQL database called **`vep_homework`**
   * Commits in batches of **≥ 500 rows** to avoid huge transactions

3. **Run and verify**
   * Execute the script to load `HG001.json`, `HG002.json`, and `HG003.json` into the database
   * Verify loaded data with SQL queries, for example:
   ```sql
   -- Show variants in TP53 across all samples
   SELECT sample_name, symbol, genotype, impact, chrom, pos
     FROM variant_call vc
     JOIN variant v USING (variant_id)
     JOIN gene g USING (gene_id)
    WHERE symbol = 'TP53'
    ORDER BY sample_name, pos;
   
   -- Summary statistics
   SELECT sample_name, COUNT(*) AS variant_count
     FROM variant_call
    GROUP BY sample_name;
   ```

## Hints & Best Practices

### 🎯 Transcript Selection Strategy
* **Primary rule**: Choose `canonical: 1` transcripts when available
* **Fallback rule**: If no canonical transcript, select by worst impact using this hierarchy:
  * **HIGH** (stop_gained, frameshift_variant, splice_acceptor_variant, splice_donor_variant)
  * **MODERATE** (missense_variant, protein_altering_variant)  
  * **LOW** (synonymous_variant, start_retained_variant)
  * **MODIFIER** (downstream_gene_variant, upstream_gene_variant, intron_variant)

### 🔧 Technical Implementation
* **Database driver**: Use `psycopg2-binary` for PostgreSQL connectivity
* **Bulk operations**: Use `session.bulk_insert_mappings()` or `session.bulk_save_objects()` for performance
* **Conflict handling**: Consider `INSERT ... ON CONFLICT DO NOTHING` for genes/variants that appear in multiple samples
* **Memory management**: Process files sequentially to avoid loading all data into memory at once

### 📊 Data Parsing Tips
* **VCF input parsing**: Split on `\t`, handle FORMAT:SAMPLE structure in columns 8-9
* **JSON structure**: Each file is an array `[{variant1}, {variant2}, ...]`
* **Sample naming**: Extract from filename using `Path(filename).stem`
* **Missing values**: Convert `"."` to `None/NULL`, handle absent dictionary keys gracefully

### 🏥 Error Handling
* **Malformed data**: Skip variants with missing required fields, log warnings
* **Database constraints**: Handle unique constraint violations for duplicate variants
* **Transaction batching**: Commit every 500-1000 records to balance performance and rollback granularity

### 🧪 Testing Strategy
* **Start small**: Test with a subset of variants first
* **Verify relationships**: Ensure foreign keys reference existing records
* **Data validation**: Check that sample names, coordinates, and impacts are correctly extracted


---

### 🔧 Advanced challenge (optional)

Create an additional **`transcript`** table and model the full *gene – transcript – variant* hierarchy.  
Populate it with all transcripts referenced under `transcript_consequences`. Pay special attention to the **canonical** flag.


---

## Analysis Notes

### 🔍 Key Data Extraction:

1. **VCF Input Parsing**: 
   ```
   "input": "chr1\t14599\t.\tT\tA\t44.3\tPASS\t.\tGT:GQ:DP:AD:VAF:PL\t1/1:5:3:1,2:0.666667:42,3,0"
   ```
   - Extract: `chrom` (chr1), `pos` (14599), `ref` (T), `alt` (A)
   - Sample data: GT=1/1, DP=3 from format field
   - Sample name: From filename (HG001, HG002, HG003)

2. **Transcript Selection**:
   - Choose `canonical: 1` first, then by impact severity
   - Impact ranking: HIGH > MODERATE > LOW > MODIFIER
   